In [1]:
import importer
import numpy as np
import numpy.random as rand
import vertices_processor as vrr

importing Jupyter notebook from vertices_processor.ipynb
importing Jupyter notebook from triangle_renderization.ipynb


In [2]:
def randomSign():
    return rand.randint(low=0, high=2, size=(1, 1))*2 - 1

In [3]:
def containMutation(mutation,limits):
    if mutation < limits[0]:
        mutation = limits[0]
    elif mutation > limits[1]:
        mutation = limits[1]
    return mutation

In [4]:
def mutateCoordinate(toMutate, limits, avoid, mutateAmount):
    
    # high is a non inclusive limit, thus we have to enter the next biggest integer to include the desired limit 1
    sign = randomSign()

    mutation = toMutate + sign*mutateAmount
    mutation = containMutation(mutation,limits) # keep the mutated number inside limits
    
    # check if mutation obtained must be avoided (doing this now bcs maybe you dont want the inclusive limits to be included (?))
    if mutation in avoid:
        # if it do is an unwanted number then repeat process with different sign
        mutation = toMutate - sign*mutateAmount
        mutation = containMutation(mutation,limits) # keep the mutated number inside limits

    # if mutation is still something to be avoided, return original number
    if mutation in avoid:
        return toMutate
    else:
        return mutation
    
def maybeMutateCoordinate(toMutate, limits, avoid, mutateAmount, mutateChance):
    rnd = rand.rand()

    if rnd < mutateChance:
        return mutateCoordinate(toMutate, limits, avoid, mutateAmount)
    else:
        return toMutate
    
# for passing non vectorized parameters solution was grabbed from this link
# https://stackoverflow.com/questions/33587610/vectorize-accelerate-numpy-function-with-two-arguments-of-different-dimensions
# also, check out the documentation
# https://docs.scipy.org/doc/numpy/reference/generated/numpy.vectorize.html
maybeMutateCoordinateVectorized = np.vectorize(maybeMutateCoordinate, excluded = [1,2,3,4])



In [5]:
def mutateLongVertices(longXY, m, n, mutateChance, mutateAmount):
    while True:
        
        avoid = []
                
        X = longXY[0:3]
        limits = [0,n-1]
        X = maybeMutateCoordinateVectorized(X, limits, avoid, mutateAmount, mutateChance)
        
        Y = longXY[3:6]
        limits = [0,m-1]

        Y = maybeMutateCoordinateVectorized(Y, limits, avoid, mutateAmount, mutateChance)
        
        XY = np.column_stack((X,Y))
        
        if ~vrr.repeatedVertices(XY):
            break
    
    XY = np.reshape(XY, 6, order='F')
    
    return XY


def mutateMultipleLongVertices(longXYs, m, n, mutateChance, mutateAmount):
    numVertices = longXYs.shape[1]
    
    for k in range(numVertices):
        longXY = longXYs[:,k]
        longXY = mutateLongVertices(longXY, m, n, mutateChance, mutateAmount)
        longXYs[:,k] = longXY
        
    return longXYs
            

In [41]:
def mutateGray(gray, mutationChance, mutationPercentage):
    
#     https://stackoverflow.com/questions/38860095/create-random-numpy-matrix-of-same-size-as-another
#     https://docs.scipy.org/doc/numpy/reference/generated/numpy.random.rand.html
    chancesArr = np.random.rand(*gray.shape)

#     https://stackoverflow.com/questions/19766757/replacing-numpy-elements-if-condition-is-met
    mutationArr = (chancesArr < mutationChance).astype(int)
    
    (M,) = np.where(mutationArr == 1)
    
# https://docs.scipy.org/doc/numpy/reference/generated/numpy.where.html    
    for k in M:
        gray[k] = gray[k] + randomSign()*gray[k]*mutationPercentage
        
    gray = np.where(gray > 0, gray, 0)
    gray = np.where(gray < 1, gray, 1)
    
    return gray
